In [1]:
import sys
!{sys.executable} -m pip install psycopg2
import psycopg2

#init crediential storage
userName = None
passWord = None

#init report vars
er_salaries = None
er_inv_cost = None
ir_orderNumber = None
ir_modelID = None
ir_customerID = None
cr_mNumber = None
cr_quantitySold = None
cr_customerID = None
rr_saleValue = None
rr_employeeID = None
rr_customerID = None

def main():
    conn = None
    userName = input("Username: ")
    passWord = input("Password: ")
    
    try:
        # connect to PostgreSQL
        conn = psycopg2.connect(host="localhost",database="testdb", user=userName, password=passWord)
        stay = True
        while stay:
            instr = split(lower(input("> ")))
            switcher = {
                "create_roles" : create_roles(),
                "create_user" : create_user(instr[1],instr[2])
                
            }
            
            commands = switcher.get(instr[0], "Invalid Command\n")
            
            cur = conn.cursor()
        
            for command in commands:
                cur.execute(command)
            
            cur.close()
            conn.commit()

    #Handle exceptions    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    #Ensure closed
    finally:
        if conn is not None:
            conn.close()
            
def create_roles():
    return ("""
        CREATE ROLE admin;
        ""","""
        CREATE ROLE sales_person;
        ""","""
        CREATE ROLE engineer;
        ""","""
        CREATE ROLE HR;
        """)
    
def create_user (user_name): #https://pythontic.com/database/mysql/create%20user
    return ("""
        CREATE USER 'user_name'@'localhost' IDENTIFIED BY 'pass_word';
        """)

def create_views():
    return ("""
        CREATE engineerView as
            select firstName, lastName /*title?*/
            from Employee
        """)    

def grant_privs (role, user_name):
    return ("""
        GRANT ADMIN to admin;
        ""","""
        GRANT CREATE TABLE to admin with grant option;
        ""","""
        GRANT INSERT on employee to admin with grant option;
        ""","""
        GRANT SELECT, UPDATE on customer to sales_person;
        ""","""
        GRANT INSERT on order to sales_person;
        ""","""
        /*access sales reports*/
        ""","""
        GRANT SELECT, UPDATE on model to engineer;
        ""","""
        GRANT SELECT UPDATE on inventory to engineer;
        ""","""
        GRANT SELECT on engineerView;
        ""","""
        GRANT SELECT, UPDATE on employee to HR"""
        )
def setup_tables ():
    return ( """
        CREATE TYPE stENUM AS ENUM ('Hourly', 'Salary');
        ""","""
        CREATE TYPE pENUM AS ENUM ('Hourly', 'Salary');
        ""","""
        create table customer (
        customerId int,
        firstName varchar(20),
        lastName varchar(20),
        Primary key (customerID)
        );
        ""","""
        create table composes(
        ID int,
        mNumber	int,
        foreign key (ID) references Inventory(ID),
        foreign key (mNumber) references Model(mNumber),
        Primary key (ID, mNumber)
        );
        ""","""
        create table inventory(
        modelID int,
        cost numeric(8,2),
        leadTime interval,
        inStock int,
        category varchar(32),
        foreign key (modelID) references model(mNumber)
        primary key (modelID)
        );
        ""","""
        Create table Employee (
        firstName varchar(32),
        lastName varchar(32),
        ssn int,
        employeeID int,
        Salary int,
        salaryType stENUM,
        userID int,
        priviledge pENUM,
        loginTime timestamp,
        logoutTime timestamp,
        Primary key (employeeID)
        );
        ""","""
        Create table model(
        mNumber int,
        salePrice numeric(8,2),
        Primary key (mNumber)
        );
        ""","""
        create table orders (
        orderNumber int,
        quantitySold int,
        saleValue int,
        Primary key (orderNumber),
        employeeID int,
        customerID int,
        mNumber int,
        Foreign key (employeeID) references Employee(employeeID),
        Foreign key (customerID) references Customer(customerID),
        Foreign key (mNumber) references Model(mNumber)
        );
        """
    )
            

def build_rev_report(orderID):
    return "SELECT saleValue, employeeID, customerID from Orders"
        
        #rr_saleValue = row[0]
        #rr_employeeID = row[1]
        #rr_customerID = row[2]


def build_cust_report(cID): 
        return "SELECT mNumber, SUM(quantitySold), customerID FROM order WHERE customerID = %s GROUP BY mNumber"
        
        #cr_mNumber = row[0]
        #cr_quantitySold = row[1]
        #cr_customerID = row[2]

def build_inv_report():
    return "SELECT orderNumber, modelID, customerID FROM order WHERE customerID = %s GROUP BY mNumber"
        
        rows = cur.fetchall()
        #for row in rows:
            #ir_orderNumber = row[0]
            #ir_modelID = row[1]
            #ir_customerID = row[2]
            
def build_expense_report():
    #Expense report, employee showing salary, bonus expense and part cost
    #All expenses included in Employee.Salary_Type = Salary & Inventory.Cost
    return ("SELECT SUM(Salary) FROM Employee WHERE Salary_Type = 'Salary'", "SELECT SUM(Cost) FROM Inventory")
        
        er_salaries = cur.fetchone()
        er_inv_cost = cur.fetchone()

    
def view_rev_report():
    if rr_employeeID is not None:
        print("Employee %s was responsible for $ %s in revenue from customer %s" % (rr_employeeID, rr_saleValue, rr_customerID))
    else:
        print("Revenue report not yet build, please contact admin for report generation")
        
def view_cust_report():
    if cr_customerID is not None:
        print("Customer %s purchased %s units of model %s" % (cr_customerID, cr_numSold, cr_mNumber ))
    else:
        print("Customer report not yet build, please contact admin for report generation")
        
def view_inv_report():
    if ir_orderNumber is not None:
        print("Order %s requires part %s of which there are %s in stock" % (ir_orderNumber, ir_modelID, ir_customerID ))
    else:
        print("Inventory report not yet build, please contact admin for report generation")
        
def view_expense_report():
    if er_salaries is not None:
        f"The annual cost of salaried employees is ${er_salaries} and the total cost of recorded inventory is {er_inv_cost}"
    else:
        print("Expense report not yet build, please contact admin for report generation")
    

def view_customer(customer_id):
    return """SELECT * FROM customer
             WHERE customer_id = customer.customer_id;"""


def update_customer(customer_id, firstName, lastName): #is this method for updating existing customers or inserting new ones?
    return """INSERT into customer VALUES(%d, %s, %s);"""%(customer_id, firstname, lastName)
    

def create_order(orderNumber, quantity, total, empID, custID, model_id): #Will order number be randomly generated? Or will we utilize a sequential counter?
    return """INSERT into orders (orderNumber, quantitySold, saleValue, employeeID, customerID, mNumber) 
             VALUES(%d, %d, %d, %d, %d, %d);"""%(orderNumber, quantity, total, empID, custID, model_id) #shouldn't the saleValue attribute be a numeric?
    

def access_model(model_id): #Is this correct? No, theres no model_id attribute for customer
    return """SELECT salePrice FROM customer
             WHERE model_id = model.mNumber;"""
    
    

def update_model(model_id, price): #Is this correct? 
    return """UPDATE model
             SET salePrice = %8.2f;"""%(price)
    


def access_inventory():
    return """SELECT * FROM inventory;"""

def update_inventory(): #what do we update? Add new items presumably

def view_employees_limited():
    return """SELECT firstName, lastName, employeeID FROM employee;"""
    
def access_employee(employee_id):
    return """SELECT * FROM employee WHERE %d = employeeID;"""%(employee_id)
   
def update_employee(employee_id): #what do we update?
    return """UPDATE employee
             /*__What to update?__*/
             WHERE %d = employeeID;"""%(employee_id)
   

def view_employee_sales(employee_id): #Do we do an aggregate on the orders table and sum up the sales total for that employee? Or do we list all the rows of orders that that employee created?
    return """SELECT * FROM orders WHERE %d = employeeID;"""%(employee_id)
    

IndentationError: unexpected indent (<ipython-input-1-a331da7738b0>, line 154)